In [6]:
# Loading Libraries
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

import log_hyperu as hyperu

In [51]:
# Simulate Data
torch.manual_seed(1234)  # For reproducibility

## Data Set 1 - Basic Coefficients - Normal Sample Size
variables = 4
sample = 100
true_coefs = torch.tensor([[0.5],[0.2],[5.3],[4.2]])
X = torch.randn(sample, variables)
Y = X @ true_coefs + torch.randn(sample, 1) * 0.1 

# Basic OLS Estimate
beta_OLS = torch.linalg.lstsq(X, Y).solution
print(beta_OLS)


tensor([[0.4938],
        [0.1951],
        [5.2891],
        [4.2099]])


In [104]:
## LASSO Implementation
class LassoRegression(nn.Module):
    def __init__(self, num_features):
        super(LassoRegression, self).__init__()
        self.linear = nn.Linear(variables, 1) 
        self.l1_penalty = nn.L1Loss(reduction='sum')

    def forward(self, x):
        return self.linear(x)

def lasso_regression(X, y, lambda_l1, num_epochs=1000, lr=0.01):
    # Initialize the model
    model = LassoRegression(X.shape[1])

    # Define optimizer
    optimizer = optim.SGD(model.parameters(), lr=lr)

    # Training loop
    for epoch in range(num_epochs):
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(X)
        
        # Compute L1 regularization term
        l1_reg = torch.tensor(0., requires_grad=True)
        for param in model.parameters():
            l1_reg = l1_reg + torch.norm(param, 1)

        # Compute loss
        loss = nn.MSELoss()(outputs, y) + lambda_l1 * l1_reg

        # Backward pass
        loss.backward()
        
        # Update parameters
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    # Return the trained model
    return model

trained_model = lasso_regression(X, Y, 4)
coefficients = trained_model.linear.weight.detach().numpy()
intercept = trained_model.linear.bias.item()

print("Coefficients:", coefficients)
print("Intercept:", intercept)


Epoch [100/1000], Loss: 30.822105407714844
Epoch [200/1000], Loss: 30.587779998779297
Epoch [300/1000], Loss: 30.495723724365234
Epoch [400/1000], Loss: 30.590938568115234
Epoch [500/1000], Loss: 30.5347900390625
Epoch [600/1000], Loss: 30.57209014892578
Epoch [700/1000], Loss: 30.573200225830078
Epoch [800/1000], Loss: 30.569324493408203
Epoch [900/1000], Loss: 30.54937744140625
Epoch [1000/1000], Loss: 30.59555435180664
Coefficients: [[-0.01088759 -0.01968116  3.194002    2.367281  ]]
Intercept: 0.007841205224394798


In [114]:
# Triple-Gamma-Regularization

class CustomRegularizationModel(nn.Module):
    def __init__(self, num_features):
        super(CustomRegularizationModel, self).__init__()
        self.linear = nn.Linear(variables, 1)

    def forward(self, x):
        return self.linear(x)

def custom_regularization_loss(outputs, targets, coefficients, penalty, a, c, kappa):
    phi = (2*c)/((kappa**2)*a)
    # Squared loss
    squared_loss = nn.MSELoss()(outputs, targets)
    
    # Penalty term
    penalty_term = penalty * torch.norm(coefficients, p=1)
    
    penalty_loss = 0
    for coefs in coefficients[0]:
        beta = torch.tensor([[coefs.item()]])
        loss_individual = hyperu.log_hyperu(torch.tensor([[c+0.5]]),torch.tensor([[1.5-a]]),(beta**2)/(2*phi))
        penalty_loss = penalty_loss + loss_individual
    
    # Combined loss
    loss = squared_loss - penalty_loss
    
    return loss

def train_model_with_regularization(X, y, penalty, a, c, kappa, num_epochs=1000, lr=0.01):
    model = CustomRegularizationModel(X.shape[1])
    optimizer = optim.SGD(model.parameters(), lr=lr)

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X)
        loss = custom_regularization_loss(outputs, y, model.linear.weight, penalty, a, c, kappa)
        loss.backward()
        
        #Gradient Clipping???
        nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        if (epoch+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    return model


trained_model2 = train_model_with_regularization(X, Y, 4, 0.1, 0.1, 2) # Covariates, Targets, Penalty, a, c, kappa
coefficients = trained_model2.linear.weight.detach().numpy()
intercept = trained_model2.linear.bias.item()

print("Coefficients General Function:", coefficients)
print("Intercept General Function:", intercept)

Epoch [100/1000], Loss: 31.113710403442383
Epoch [200/1000], Loss: 22.55941390991211
Epoch [300/1000], Loss: 14.998287200927734
Epoch [400/1000], Loss: 9.139435768127441
Epoch [500/1000], Loss: 5.041503429412842
Epoch [600/1000], Loss: 2.444061279296875
Epoch [700/1000], Loss: -0.793156623840332
Epoch [800/1000], Loss: 1.2230231761932373
Epoch [900/1000], Loss: 1.4403553009033203
Epoch [1000/1000], Loss: 1.4940588474273682
Coefficients General Function: [[0.49410915 0.1903951  5.2835255  4.2086163 ]]
Intercept General Function: -0.007902863435447216


In [88]:
Custom Loss Test
print(coefficients)
a = torch.tensor([[0.5]])
b = torch.tensor([[0.5]])
c = torch.tensor([[0.5]])
custom_loss = 0

for coefs in coefficients[0]:
    beta = torch.tensor([[coefs]])
    loss_individual = hyperu.log_hyperu(a,b,beta**2)
    print(loss_individual)
    custom_loss = custom_loss + loss_individual

print(custom_loss)
#print(hyperu.log_hyperu(a,b,c))

[[-0.02184849  0.01860901  3.1137912   2.3287895 ]]
tensor([[0.5479]])
tensor([[0.5515]])
tensor([[-1.1820]])
tensor([[-0.9224]])
tensor([[-1.0050]])


In [123]:
a = 0.1
c = 0.1
kappa = 2
phi = (2*c)/((kappa**2)*a)
coef = torch.tensor([[4]])

hyperu.log_hyperu(torch.tensor([[c+0.5]]),torch.tensor([[1.5-a]]),(coef**2)/(2*phi))

tensor([[-1.6707]])